# Host model on a grid node

<h2>Import dependencies</h2>

In [ ]:
import pickle
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import grid as gr
import torch.nn.functional as F
import torch.optim as optim
import syft as sy
from torch.utils.data import TensorDataset, DataLoader
import time

<h2>Setup config</h2>
Define Config parameters, init hook, etc...

In [ ]:
hook = sy.TorchHook(torch)
class Parser:
    """Parameters for training"""
    def __init__(self):
        self.epochs = 1
        self.lr = 0.001
        self.test_batch_size = 8
        self.batch_size = 8
        self.log_interval = 10
        self.seed = 1
    
args = Parser()

torch.manual_seed(args.seed)
kwargs = {}

<h2>Load dataset</h2>

In [ ]:
test_loader  = torch.utils.data.DataLoader(
        datasets.MNIST(
            "./data",
            train=False,
            download=True,
            transform=transforms.Compose(
                [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
            ),
        ),
        batch_size=args.test_batch_size,
        shuffle=False,
        drop_last=False,
    )
(data, target) = test_loader.__iter__().next()

<h2>Define Model</h2>

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

<h2>Connect with remote workers</h2>
<strong><me>Before this step, it is necessary to initialize the workers separately<me></strong>

To run an app, go to app/websocket folder and run: 
    
```python websocket_app.py --id=bob --port=4000```

This will start a websocket application on port 4000 with worker id bob.
    

In [64]:
bob = gr.WebsocketGridClient(hook, "http://localhost:4000", id="bob")
bob.connect()

In [ ]:
model = Net()
traced_model = torch.jit.trace(model, data)

In [ ]:
traced_model(data)

## Serve model

In [53]:
# list the models bob currently has
bob.models

['first_model', 'first_model1']

In [67]:
# sending the model to bob for serving on the server
bob.serve_model(traced_model, model_id="first_model")

'{"success": "Model deployed with id first_model"}'

In [59]:
# now bob.models will return our newly sent model
bob.models

['first_model', 'first_model1']

In [66]:
# deletes the remote model
bob.delete_model('first_model')

'{"status": "success"}'

In [63]:
# now the model list will be empty again
bob.models

['first_model', 'first_model1']

In [62]:
# sending a model to the worker and running inference remotely
bob.serve_model(traced_model, model_id="first_model")
bob.run_inference(model_id="first_model", data=torch.zeros(1, 1, 28, 28))

{'prediction': [[-2.322812080383301,
   -2.303769111633301,
   -2.293975830078125,
   -2.3327364921569824,
   -2.306673049926758,
   -2.274453639984131,
   -2.29142165184021,
   -2.3145298957824707,
   -2.3104403018951416,
   -2.2766435146331787]]}